In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.stats import kurtosis, skew

In [3]:
data= []

time = []

curr_X = []
curr_Y = []

delta_X = []
delta_Y = []

delta_time = []

disp = []

nonX = []
nonY = []
nonS = []


for i in range(2,3):
    participant = i
    zeros = 4 - len(str(participant))
    data.append(pd.read_csv("data\Participant"+ zeros*'0' + str(participant) + ".tsv",sep="\t"))


    time.append(data[-1]['Eyetracker timestamp'])
    curr_X.append(data[-1]['Gaze point X'])
    curr_Y.append(data[-1]['Gaze point Y'])
    delta_X.append(np.diff(curr_X[-1]))
    delta_Y.append(np.diff(curr_Y[-1]))
    delta_time.append(np.diff(time[-1]))
    disp.append(np.sqrt(np.diff(curr_X[-1])**2 + np.diff(curr_Y[-1])**2))
    #speed.append(disp[-1]/delta_time[-1])

    #remove NaN/missing
    #nonX.append(d_X[-1][np.logical_not(np.isnan(d_X[-1]))])
    #nonY.append(d_Y[-1][np.logical_not(np.isnan(d_Y[-1]))])
    #nonS.append(d_S[-1][np.logical_not(np.isnan(d_S[-1]))])


In [4]:
test = pd.read_csv("data\Participant"+ zeros*'0' + str(participant) + ".tsv",sep="\t")

In [7]:
x_val = test.iloc[:, 39:40].values
y_val = test.iloc[:, 40:41].values

print(len(x_val))
print(len(y_val))

46556
46556


In [ ]:
# Feature Duration

print("Timestamps")
#print(fix_timestamp)

fix_duration_diff = []

#Print diff in timestamps for easy feature seperation

for i in range(len(fix_timestamp)-1):
    
    fix_duration_diff.append(fix_timestamp[i+1] - fix_timestamp[i])

#print(fix_duration_diff)
#print(len(fix_duration_diff))

# Feature Displacement

fix_point_disP = []  
x_point_fixLen = []
y_point_fixLen = []
    
for i in range(len(x_fix)-1):
    
    x_point_fixLen.append(x_fix[i+1] - x_fix[i])
    
    y_point_fixLen.append(y_fix[i+1] - y_fix[i])
        
    fix_point_disP.append(np.sqrt(x_point_fixLen[i]**2 + y_point_fixLen[i]**2))

#Define boundaries for feature seperation (a certain amount of time needs to pass for a feature to be "ended")

fix_duration_lengths = []
fix_disp_lengths = []

fix_feat_disp = 0 

fix_count = 0
fix_duration_set = 0


fix_disP = []  
x_fixLen = []
y_fixLen = []
    
for i in range(len(x_fix)-1):
    
    x_fixLen.append(x_fix[i+1] - x_fix[i])
    
    y_fixLen.append(y_fix[i+1] - y_fix[i])
        
    fix_disP.append(np.sqrt(x_fixLen[i]**2 + y_fixLen[i]**2))
    
    
fix_theta_np = []

ang_time_f2 = []

fix_duration_diff_corr = []

for i in range(len(fix_duration_diff)-1):
    vec1 = [np.array(x_fixLen[i]).sum(axis=0), np.array(y_fixLen[i]).sum(axis=0)]
    vec2 = [np.array(x_fixLen[i+1]).sum(axis=0), np.array(y_fixLen[i+1]).sum(axis=0)]
 
    unit1 = vec1 / np.linalg.norm(vec1)
    unit2 = vec2 / np.linalg.norm(vec2)
    dot_product = np.dot(unit1, unit2)
    #print(np.arccos(dot_product))
    
    if (math.isnan(np.arccos(dot_product))):
        pass
    else:
        fix_theta_np.append(math.pi - np.arccos(dot_product))
        ang_time_f2.append(fix_timestamp[i+1]/1000000)
        fix_duration_diff_corr.append(fix_duration_diff)
        
print(len(fix_theta_np))

print(len(fix_duration_diff_corr))

#print(fix_duration_diff)
        
#print(len(fix_duration_diff_corr))

fix_theta_np_avg = 0

fix_feat_theta_avg = []
        
for i in range (len(fix_duration_diff_corr)):
    
    if (fix_duration_diff[i] < 100000):
        fix_count = fix_count + 1
        #count up individual lengths
        
        if (fix_duration_diff[i] < 0):
            pass
        else:
            fix_duration_set = fix_duration_set + fix_duration_diff[i]
            fix_feat_disp = fix_feat_disp + fix_point_disP[i]
            fix_theta_np_avg = fix_theta_np_avg + fix_theta_np[i]
    
        #print(fix_duration_diff[i])
        
    else:
        
        if (fix_count <= 1):
            #discard
            #print(fix_count)
            fix_count = 0 #reset
            fix_duration_set = 0
            fix_feat_disp = 0
            
        else:
            #append
            #print(fix_count)
            fix_duration_lengths.append(fix_duration_set)
            fix_duration_set = 0
            fix_disp_lengths.append(fix_feat_disp)
            fix_feat_theta_avg.append(fix_theta_np_avg/fix_count)
            fix_theta_np_avg = 0
            fix_feat_disp = 0
            fix_count = 0 #reset 
        
print(len(fix_feat_theta_avg))
        
plt.hist(fix_feat_theta_avg)
plt.show()
        
#print(count)
    
#print(len(fix_duration_lengths))

#print(fix_duration_lengths)

#When features "transition" or "end", append total time of that feature (sum of the time differences for each point in a fixation or saccade group)
    #Note: you may have to delete the first summation (deduct first point from itself)
    #A counter will be necessary to "reset" fixations, or omit singular isolated points classified as fixations
    
#Plot hisogram of each list, with their contained feature durations

#print(fix_duration_lengths)


In [ ]:
#Velocity

fix_feat_velo = []
    
for i in range(len(fix_disp_lengths)):
        
    fix_feat_velo.append(fix_disp_lengths[i]/(fix_duration_lengths[i]/1000000))


plt.scatter(np.array(np.array(fix_duration_lengths)/1000000).sum(axis=1),fix_feat_velo, c = 'red')
plt.xlabel("Fixation Feature Duration")
plt.ylim(0, 25000)
plt.xlim(0, 3.5)
plt.ylabel("Velocity")
plt.title("Velocity v. Time (Fixation Features)")
plt.savefig("Fixations_feat_velo")
plt.show()


In [ ]:

x_fix = []
y_fix = []

x_sac = []
y_sac = []

t_stamps = test.iloc[:, 0:1].values

#print(t_stamps)

sac_timestamp = []
fix_timestamp = []

leg = int(len(f_type))
print(leg)

for i in range (leg):
    if f_type[i] == 'Fixation':
        x_fix.append(x_val[i])
        y_fix.append(y_val[i])
        fix_timestamp.append(t_stamps[i])
        
    elif f_type[i] == 'Saccade':
        x_sac.append(x_val[i])
        y_sac.append(y_val[i])
        sac_timestamp.append(t_stamps[i])
        
    else:
        pass
    
print("X/Y Fixation Points")

#print(x_fix)
#print(y_fix)

print("X/Y Saccade Points")

#print(x_sac)
#print(y_sac)


        